In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import DataLoader
from torch.autograd import Variable

import math, random, sys
import numpy as np
import argparse
from argparse import Namespace
from collections import deque
import pickle as pickle

from jtnn import *
from auxiliaries import build_parser, set_random_seed, get_args_and_vocab
import rdkit
import json, os
from rdkit import RDLogger
import pathlib

In [2]:
lg = RDLogger.logger() 
lg.setLevel(RDLogger.CRITICAL)
root = str(pathlib.Path().absolute())

## Arguments 

In [3]:
# overwrite default parameters
cmd_args = {
    'beta': 0.002, 
    'max_beta': 1.0,
    'latent_size': 4,
    'target': 'splitting',
}

train_path = os.path.join(root, 'pred_processed', cmd_args['target'], 'train')
val_path = os.path.join(root, 'pred_processed', cmd_args['target'], 'val')

In [4]:
model_dir = os.path.join(root, 'pred_models')
json_path = os.path.join(model_dir, 'default_pred_args.json')
with open(json_path) as handle:
    arguments = json.loads(handle.read())
arguments.update(cmd_args)
if 'seed' in arguments:
    set_random_seed(arguments['seed'])
else:
    arguments['seed'] = set_random_seed()
arguments['cuda'] = torch.cuda.is_available()
arguments

{'anneal_iter': 1000,
 'anneal_rate': 0.9,
 'batch_size': 32,
 'beta': 0.002,
 'clip_norm': 50.0,
 'cuda': True,
 'depthG': 3,
 'depthT': 20,
 'epoch': 50,
 'hidden_size': 400,
 'kl_anneal_iter': 2000,
 'latent_size': 4,
 'load_epoch': 0,
 'lr': 0.0003,
 'max_beta': 1.0,
 'n_out': 1,
 'num_layers': 5,
 'plot': True,
 'print_iter': 100,
 'save_iter': 100000,
 'seed': 2595467008,
 'step_beta': 0.002,
 'total_trials': 20,
 'use_activation': True,
 'vocab': './data/rafa/vocab.txt',
 'target': 'strength'}

In [5]:
def train(parametrization):
    global arguments, model_dir
    args = {**arguments, **parametrization}
    model_name = "{}-h{}-l{}-n{}-e{}-s{}".format(
        args['target'], args['hidden_size'], args['latent_size'],
        args['num_layers'], args['epoch'], args['seed']
    )
    args['save_dir'] = os.path.join(model_dir, model_name)
    # save model settings
    os.makedirs(args['save_dir'], exist_ok=True)
    dump_json_path = os.path.join(args['save_dir'], 'model.json')
    if not os.path.exists(dump_json_path):
        with open(dump_json_path, "w") as fp:
            json.dump(args, fp, sort_keys=True, indent=4)
    args = Namespace(**args)
    vocab = Vocab([x.strip("\r\n ") for x in open(args.vocab)])
    print(args)
    model = RAFAVAE(vocab, args, evaluate=False)
    if args.cuda:
        model = model.cuda()
    return trainer(model, args, vocab)

In [6]:
def trainer(model, args, vocab):
    global train_path, val_path
    for param in model.parameters():
        if param.dim() == 1:
            nn.init.constant_(param, 0)
        else:
            nn.init.xavier_normal_(param)

    if args.load_epoch > 0:
        model.load_state_dict(torch.load(args.save_dir + "/model.iter-" + str(args.load_epoch)))

    print(("Model #Params: %dK" % (sum([x.nelement() for x in model.parameters()]) / 1000,)))

    optimizer = optim.Adam(model.parameters(), lr=args.lr)
    scheduler = lr_scheduler.ExponentialLR(optimizer, args.anneal_rate, verbose=True)
    # As per warning
    # scheduler.step()

    param_norm = lambda m: math.sqrt(sum([p.norm().item() ** 2 for p in m.parameters()]))
    grad_norm = lambda m: math.sqrt(sum([p.grad.norm().item() ** 2 for p in m.parameters() if p.grad is not None]))

    total_step = args.load_epoch
    meters = np.zeros(1)

    for epoch in range(args.epoch):
        loader = MolTreeFolder(train_path, vocab, args.batch_size, num_workers=4)
        val_loader = MolTreeFolder(val_path, vocab, args.batch_size, num_workers=4)
        model.train()
        for batch in loader:
            total_step += 1
            model.zero_grad()
            loss = model(batch)
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), args.clip_norm)
            optimizer.step()

            meters = meters + np.array([loss.data.cpu().numpy()])

            if total_step % args.print_iter == 0:
                meters /= args.print_iter
                print(("[%d] Loss: %.2f, PNorm: %.2f, GNorm: %.2f" % (total_step, meters[0], param_norm(model), grad_norm(model))))
                sys.stdout.flush()
                meters *= 0

            if total_step % args.save_iter == 0:
                torch.save(model.state_dict(), args.save_dir + "/model.iter-" + str(total_step))

            if total_step % args.anneal_iter == 0:
                scheduler.step()
                print(("learning rate: %.6f" % scheduler.get_last_lr()[0]))

        val_loss = 0.0
        val_steps = 0
        model.eval()
        for val_batch in val_loader:
            val_steps += 1
            loss = model(val_batch)
            val_loss += float(loss.data)

        print(("Validation Loss: %.6f" % (val_loss / val_steps)))
        sys.stdout.flush()
    
    torch.save(model.state_dict(), args.save_dir + f"/model")
    plot_train(model, args)
    return model


In [7]:
def plot_train(model, args):
    vocab = Vocab([x.strip("\r\n ") for x in open(args.vocab)])
    model.set_mode(evaluate=True)
    model.eval()
    loader = MolTreeFolder(train_path, vocab, args.batch_size, num_workers=4)
    pred = np.zeros(0)
    act = np.zeros(0)
    for batch in loader:
        output = model(batch)
        output = output.data.cpu().numpy().squeeze()
        labels = np.array([x.label for x in batch[0]])
        pred = np.concatenate((pred, output))
        act = np.concatenate((act, labels))
    save_dir = args.save_dir
    np.save(os.path.join(save_dir, 'pred-train'), pred)
    np.save(os.path.join(save_dir, 'act-train'), act)
    import matplotlib
    matplotlib.use('Agg')
    import matplotlib.pyplot as plt
    pred, act = np.load(os.path.join(save_dir, 'pred-train.npy')), np.load(os.path.join(save_dir, 'act-train.npy'))
    plt.figure()
    ax = plt.gca()
    ax.plot([0,1],[0,1], transform=ax.transAxes, color='green')
    ax.set_aspect('equal')
    plt.scatter(act, pred, s=1)
    plt.xlabel('Actual')
    plt.ylabel('Predicted')
    plt.title(args.target)
    plt.savefig(os.path.join(save_dir, args.target+'-train.png'), dpi=300)


In [8]:
def train_evaluate(parametrization):
    model = train(parametrization)
    return evaluate(model)

## Hyperparameter search

In [ ]:
# hyperparameter search
from ax.service.managed_loop import optimize
best_parameters, values, experiment, model = optimize(
    parameters = [
        { "name": "lr", "type": "range", "bounds": [1e-5, 5e-3] },
        { "name": "hidden_size", "type": "range", "value_type": "int", "bounds": [100, 650] },
        { "name": "num_layers", "type": "range", "value_type": "int", "bounds": [1, 5] },
        { "name": "latent_size", "type": "range", "value_type": "int", "bounds": [40, 100] },
        { "name": "epoch", "type": "range", "value_type": "int", "bounds": [10, 20] },
    ],
    evaluation_function=train_evaluate,
    minimize=True,
    total_trials=20
)
means, covariances = values
print('best parameters:', best_parameters)
print(means)


[INFO 06-14 21:31:29] ax.modelbridge.dispatch_utils: Using Bayesian Optimization generation strategy: GenerationStrategy(name='Sobol+GPEI', steps=[Sobol for 5 trials, GPEI for subsequent trials]). Iterations after 5 will take longer to generate due to  model-fitting.
[INFO 06-14 21:31:29] ax.service.managed_loop: Started full optimization with 20 steps.
[INFO 06-14 21:31:29] ax.service.managed_loop: Running optimization trial 1...


Namespace(anneal_iter=1000, anneal_rate=0.9, batch_size=32, beta=0.002, clip_norm=50.0, cuda=True, depthG=3, depthT=20, epoch=68, hidden_size=336, kl_anneal_iter=2000, latent_size=65, load_epoch=0, lr=0.004179916833043099, max_beta=1.0, n_out=1, num_layers=3, plot=True, print_iter=100, save_dir='/home/huang651/junction-tree/pred_models/strength-h336-l65-n3-e68-s2595467008', save_iter=100000, seed=2595467008, step_beta=0.002, target='strength', total_trials=20, use_activation=True, vocab='./data/rafa/vocab.txt')
Model #Params: 2799K
Adjusting learning rate of group 0 to 4.1799e-03.
[100] Loss: 0.44, PNorm: 92.78, GNorm: 0.60
[200] Loss: 0.22, PNorm: 99.67, GNorm: 0.96
[300] Loss: 0.21, PNorm: 110.07, GNorm: 1.70
[400] Loss: 0.20, PNorm: 114.75, GNorm: 0.21
[500] Loss: 0.18, PNorm: 116.87, GNorm: 0.26
[600] Loss: 0.17, PNorm: 118.54, GNorm: 0.86
[700] Loss: 0.18, PNorm: 121.07, GNorm: 0.52
[800] Loss: 0.18, PNorm: 128.54, GNorm: 1.12
[900] Loss: 0.18, PNorm: 133.79, GNorm: 0.31
[1000] Lo